In [1]:
import datetime as dt
import time
#import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Playing around with stuff

In [3]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

[1, 2, 3, 5, 6, 7, 9, 10, 11]
[array([9, 9, 9]), array([9, 9, 9]), array([9, 9, 9])]
[9 9 9 9 9 9 9 9 9]


In [4]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [5]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [6]:
supportIndices = [0, 4]
injection = pxop.SubSample(10, supportIndices).T

x = np.ones(len(supportIndices))

print('injection:', injection(x))

injection: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
indices = np.array([k + k*3 for k in range(3)], dtype="int32")
print(indices)
print(testarr[indices])

[0 4 8]
[1 2 3]


In [54]:
dim1 = 2
dim2 = 3

l1normdim1 = pxop.L1Norm(dim=dim1)
l2normdim2 = pxop.L2Norm(dim=dim2)

x1 = np.ones(dim1)
x2 = 2*np.ones(dim2)
bigx = np.hstack([x1, x2])

#sumOp = pxop.Sum(arg_shape=)
normOp = pxop.hstack([l1normdim1, l2normdim2])
print(normOp(bigx))
print(l1normdim1(x1) + l2normdim2(x2))

[5.46410162]
[5.46410162]


In [ ]:
toyM = 4
ez = np.array([k + k*toyM for k in range(toyM)], dtype="int32")
print(ez)

ez2 = [index for k in range(toyM) for index in range(k*toyM + k + 1, k*toyM + k + toyM + 1)]
print(ez2)

### Generation of the source

In [3]:
path = "simulated_data/events13.csv"
beta = 5
hpL = HawkesLikelihood(path=path, beta=beta)
#hpL.plot_realization()

In [4]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0249
Lipschitz constant: 115.24377964750194


### Generation of the source (using multiple realizations)

In [45]:
totalProcesses = 15
beta = 5

paths = [f"simulated_data/events{i}.csv" for i in range(totalProcesses)]
allHPs = np.full(totalProcesses, None)

for i in range(totalProcesses):
    print(f"Initializing process {i+1}...")
    allHPs[i] = HawkesLikelihood(path=paths[i], beta=beta)
M = allHPs[0].M
# j = 3
# print(allHPs[j].E(np.ones(allHPs[j].E.shape[1])))

Initializing process 1...
Initializing process 2...
Initializing process 3...
Initializing process 4...
Initializing process 5...
Initializing process 6...
Initializing process 7...
Initializing process 8...
Initializing process 9...
Initializing process 10...
Initializing process 11...
Initializing process 12...
Initializing process 13...
Initializing process 14...
Initializing process 15...


In [46]:
# Define big forward operator A: R^{M + M^2} -> R^{K1 + ... + Kn}
bigA = pxop.vstack([allHPs[i].opA for i in range(totalProcesses)])

# Define big E operator E1(x_1, ..., x_K1) + E2(x_{K1+1}, ..., x_{K1+K2}) + ...
# E: R^{K1 + ... + Kn} -> R
bigE = pxop.hstack([allHPs[i].E for i in range(totalProcesses)])

# Define big -log(likelihood operator)
bigNegLogL = bigE * bigA

In [47]:
# print(bigE)
# print(bigNegLogL)
# x = np.ones(bigE.shape[1])
# print(bigE(x))

In [48]:
start = time.time()
lip = bigA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)

Computation time of the Lipschitz constant: 0.3081
Lipschitz constant: 471.8502818691068


### Defining solvers

In [56]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 5e1

In [57]:
# pfw = pyfwl.PFWLasso(
#     forwardOp=hpL.opA,
#     convexOp=hpL.E,
#     lambda_=lambda_,
#     M=hpL.M,
#     negLogL=hpL.negLogL,
#     ms_threshold=ms_threshold,
#     init_correction_prec=init_correction,
#     final_correction_prec=final_correction,
#     remove_positions=remove,
#     min_correction_steps=correction_steps,
#     show_progress=False,
# )
pfw = pyfwl.PFWLasso(
    forwardOp=bigA,
    convexOp=bigE,
    lambda_=lambda_,
    M=M,
    negLogL=bigNegLogL,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [58]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [59]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
initial value: [16431.99225718]
new indices [71]
actual new indices: 1
new indices [21 41 51 53]
actual new indices: 4
new indices [ 1  5  8 11 17 21 23 26 31 33 35 37 41 42 44 51 53 61 62 68 69 71]
actual new indices: 17
new indices [ 1  5  8 11 12 17 21 23 24 26 31 33 35 37 41 44 49 50 51 53 61 62 66 68
 69 71]
actual new indices: 5
new indices [ 1  5  8 17 21 23 26 31 33 35 37 41 44 51 53 61 62 68 69 71]
actual new indices: 0
new indices [ 1  5  8 17 21 23 26 35 41 44 51 53 61 62 68 69 71]
actual new indices: 0
new indices [ 8 21 26 35 41 44 51 53 61 68 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 41 44 51 53 69 71]
actual new i

### Evaluation of the solve

In [60]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 16.2211
Final value of objective function:
	PFW : -215220.37


In [61]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]

#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
for i in range(M):
    print(f"Process {i}:", data_p["x"][(i*(M + 1)):(i*(M + 1) + M + 1)])

# Arrange intensities in a M x M matrix
intensities = np.zeros((M, M))
for i in range(M):
    intensities[i, :] = data_p["x"][(i*(M + 1) + 1):(i*(M + 1) + M + 1)]

Process 0: [8.52283394 1.81369348 0.         0.         0.         2.11031491
 0.         0.         2.49867026]
Process 1: [8.4012962  0.         1.52299467 1.64664656 0.         0.
 0.         0.         2.33277549]
Process 2: [9.10498048 0.         0.         3.59250122 0.         2.14625727
 1.78839866 0.         2.79544619]
Process 3: [8.4656858  0.         0.         0.         1.67657947 0.
 1.95391461 0.         2.42796902]
Process 4: [9.21772692 1.76831365 0.         0.         0.         4.24649332
 1.88980259 0.         2.96175891]
Process 5: [9.10116224 0.         0.         0.         1.54386941 1.89699403
 3.79658097 0.         3.67887336]
Process 6: [8.55529222 0.         0.         0.         0.         0.
 0.         1.9231656  2.38600333]
Process 7: [9.4794496  0.         0.         1.85320832 0.         2.27866328
 3.49087439 0.         5.77384838]


In [62]:
# Visualize solution
fig, ax = plt.subplots(1, 1, figsize=(5,5))
im = ax.imshow(intensities/(beta*beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, M, M, 1])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('L1 norms of the $h_{ij}$')
plt.show()

for i in range(M):
    print(f"mu{i+1}:", data_p["x"][i*(M + 1)])

print(intensities/(beta*beta))

mu1: 8.522833943418352
mu2: 8.401296197840823
mu3: 9.104980480619759
mu4: 8.465685801621003
mu5: 9.217726924503545
mu6: 9.101162241507522
mu7: 8.555292221972485
mu8: 9.479449598914872
[[0.07254774 0.         0.         0.         0.0844126  0.
  0.         0.09994681]
 [0.         0.06091979 0.06586586 0.         0.         0.
  0.         0.09331102]
 [0.         0.         0.14370005 0.         0.08585029 0.07153595
  0.         0.11181785]
 [0.         0.         0.         0.06706318 0.         0.07815658
  0.         0.09711876]
 [0.07073255 0.         0.         0.         0.16985973 0.0755921
  0.         0.11847036]
 [0.         0.         0.         0.06175478 0.07587976 0.15186324
  0.         0.14715493]
 [0.         0.         0.         0.         0.         0.
  0.07692662 0.09544013]
 [0.         0.         0.07412833 0.         0.09114653 0.13963498
  0.         0.23095394]]


In [116]:
hpL.plot_realization()